In [258]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

np.random.seed(seed=42)

# Initial Class Definitions

In [314]:
# implements the basic features of our creatures.
# they are:
#    > gender (init=distribution)
#    > alive (init=true)
#    > age (init=distr or 0)
#    > offspring (init=0)
#    > fertility (init=distr)
class Creature:
    universe = None
    
    def __init__(self, idx, instant):
        self.gender = 'm' if (np.random.uniform(low=0.0, high=1.0) < self.universe.mf_ratio[type(self)]) else 'f'
        self.fertility = True if (np.random.uniform(low=0.0, high=1.0) < self.universe.fertility_ratio[type(self)]) else False
        self.age = np.random.randint(low=0, high=self.universe.lifespan[type(self)] + 1) if (instant == 0) else 0
        self.generation = instant
        self.idx = idx
        self.alive = True
        self.offspring = 0
    
    def is_adult(self):
        return self.age >= self.universe.adult_age[type(self)]
    
    def can_procriate(self):
        return ((self.gender == 'f') and self.fertility)
    
    def children(self):
        ncs = int(np.round(np.random.normal(loc=self.universe.offspring_mean[type(self)],
                                                    scale=self.universe.offspring_variance[type(self)])))
        return [type(self)(instant=1) for i in
                range(ncs)]
    
    def increment_age(self):
        self.age = self.age + 1

    
    # checks if the creature died randomly. If it did,
    # updates its 'alive' attribute.
    #
    # returns if a random death occured (True) or not (False)
    def random_death(self):
        if (np.random.uniform(low=0.0, high=1.0) < self.universe.random_death_chance[type(self)]):
            self.alive = False
            return True
        else:
            return False

        
    # checks if an old death occured. If it occured, updates
    # the 'alive' attribute of the creature.
    #
    # returns if an old age death actually occured (True) or
    # not (False)
    def old_age_death(self):
        if (self.age > self.universe.lifespan[type(self)]):
            self.alive = False
            return True
        else:
            return False
    
    # =(
    def die(self):
        self.alive = False
        #np.round(np.random.normal(loc=self.offspring_mean[cls], scale=offspring_variance[cls]))
        

# implements fly-only features and conditions
# <<< this one's the predator/parasite
class Fly(Creature):
    pass

# implements moth-only features and conditions
class Moth(Creature):

    def is_caterpillar():
        return ((self.universe.egg_age[type(self)] < self.age) and
                (self.age < self.universe.adult_age[type(self)]))


# set of biological laws that our system obeys,
# amongst which:
#    > distributions between male and female gender of each creature
#    > lifespan of each creature
#    > initial lifespan distribution for each creature
#    > fertility ratio for each creature
#    > offspring mean and variance (normal distributions predefined for
#      these distributions)
class Universe:
    def __init__(self, fmfr, flfsp, ffr, fom, fov, faa, frd,  # fly parameters
                 mmfr, mlfsp, mfr, mom, mov, maa, mea, mrd,   # moth parameters
                 predation_coefficient):                      # other parameters
        
        self.mf_ratio = {Fly: fmfr, Moth: mmfr}
        self.lifespan = {Fly: flfsp, Moth: mlfsp}
        self.fertility_ratio = {Fly: ffr, Moth: mfr}
        self.offspring_mean = {Fly: fom, Moth: mom}
        self.offspring_variance = {Fly: fov, Moth: mov} 
        self.adult_age = {Fly: faa, Moth: maa}
        self.egg_age = {Fly: None, Moth: mea}
        self.random_death_chance = {Fly: frd, Moth: mrd}
        
        self.predation_coefficient = predation_coefficient
    
    
class WonderfulWorld:
    
    def __init__(self, n_moths, n_flies, universe, end_of_times):
        self.universe = universe
        Creature.universe = universe
        self.instant = 0
        self.end_of_times = end_of_times
          
        # initializes:
        #    - ages based on a uniform distribution
        #    - genders following the universe's male/female ratios
        self.moths = [[Moth(i, self.instant) for i in range(n_moths)], [] * end_of_times]
        self.flies = [[Fly(i, self.instant) for i in range(n_flies)], [] * end_of_times]
        
        self.living_creatures_idx = {
            Moth: [(0,i) for i in range(n_moths)],
            Fly: [(0,i) for i in range(n_flies)]
        }
        

    # keep the caterpilar indexes
    def update_caterpillars_idx():
        pass
    
        # remove the old caterpillars
        
        # add the new caterpillars

    
    
    # OPTIMISATION: use the current creature's lifespan as
    # starting point for the lists
    def living_creatures(self, creatures):
        for c_list in creatures[:self.instant + 1]:
            for c in c_list:
                if c.alive:
                    yield c
    
    
    # save the useful data on a dataframe for each generation
    def document_step(self):
        pass
    
    def kill_creature(self, c_type, c_gen, c_idx):
        self.living_creatures_idx[c_type].remove((c_gen, c_idx))
    
    
    # calculates the chances of a predation happening. Idea: use
    # the ratio caterpillars/flies instead of the moth/flies ratio,
    # since only the caterpillars can be infested
    def predation_happens():
        pass
    
    
    # lets us check how the moths and 
    def show_data(self, instant):
        pass
    
    def single_step(self):
        # moth stuff:
        #    > see if it died randomly
        #    > see if died of old age
        #        - if it was female and fertile, procriates on death
        #    > nothing happens bean stew (increment age)
        #    > check if it's a caterpillar now (not necessary)
        #
        # >>> how to process moths from older generations??
        #     - copy the moths on the new generation list? (nah)
        #     - go checking through all generations the living moths
        #     - and process them (nah)
        #     - implement a generator method living_moths() that returns
        #       the indexes of only the living moths and iterate on that
        #       (oya)
        #
        # IMPORTANT OBS: WE DO NOT PROCESS THE NEWBORN CREATURES
        # ON THE SAME LOOP, ONLY ON THE NEXT ONE (even tho it would
        # be easy to include them, it would suffice to set the upper
        # limit on the living_creatures() generator as 'instant + 2')
        #
        for m in self.living_creatures(self.moths):
            if not m.random_death():
                if m.old_age_death():
                    self.kill_creature(Moth, m.generation, m.idx)
                    if m.can_procriate():
                        children = m.children()
                        self.moths[instant + 1].append(children)
                else:
                    m.increment_age()
            else:
                self.kill_creature(Moth, m.generation, m.idx)
        
        # fly stuff:
        #    > random death
        #    > death by old age
        #        - with its last breath, it parasited a moth
        #          (or not, we roll the dice to check)
        #    > nothing happens bean stew (increment age)
        for f in self.living_creatures(self.flies):
            if not f.random_death():
                if f.old_age_death():
                    if predation_happens():
                        pass
                    #np.random.randint
                else:
                    f.increment_age()

In [315]:
# beetle universe laws:
#     - male/female ratio (mfr)
#     - lifespan (lfsp)
#     - fertility ratio (fr)
#     - offspring mean and variance (om and ov)
#     - adult age (aa, equal or higher is adult)
#     - egg age (ea, less or equal is egg)
#     - random death (chance)
# fly universe laws parameters
fly_params = {
                'mfr': 0.5,
                'lfsp': 15,
                'fr': 0.4,
                'om': 7,
                'ov': 5,
                'aa': 9,
                'rd': 0.1
            }

# moth universe laws parameters
moth_params = {
                'mfr': 0.5,
                'lfsp': 20,
                'fr': 0.6,
                'om': 21,
                'ov': 10,
                'aa': 12,
                'ee': 4,
                'rd': 0.1
              }

# other parameters:
#     - predation coefficient (pc)
other_params = {
                'pc': 0.7
}

# universe laws definition
u = Universe(*fly_params.values(), *moth_params.values(), *other_params.values())

# initial number of beetles and wasps
nm = 200
nf = 100

# number of simulation steps
steps = 500

# world definition
w = WonderfulWorld(nm, nf, u, steps)

In [316]:
w.single_step()

In [317]:
w.universe.random_death_chance

{__main__.Fly: 0.1, __main__.Moth: 0.1}

In [270]:
w.moths[0][0].age
ages = [m.can_procriate() for m in w.flies[0]]

In [301]:
w.living_creatures_idx

{__main__.Moth: [(0, 0),
  (0, 1),
  (0, 2),
  (0, 3),
  (0, 4),
  (0, 5),
  (0, 6),
  (0, 7),
  (0, 8),
  (0, 9),
  (0, 10),
  (0, 11),
  (0, 12),
  (0, 13),
  (0, 14),
  (0, 15),
  (0, 16),
  (0, 17),
  (0, 18),
  (0, 19),
  (0, 20),
  (0, 21),
  (0, 22),
  (0, 23),
  (0, 24),
  (0, 25),
  (0, 26),
  (0, 27),
  (0, 28),
  (0, 29),
  (0, 30),
  (0, 31),
  (0, 32),
  (0, 33),
  (0, 34),
  (0, 35),
  (0, 36),
  (0, 37),
  (0, 38),
  (0, 39),
  (0, 40),
  (0, 41),
  (0, 42),
  (0, 43),
  (0, 44),
  (0, 45),
  (0, 46),
  (0, 47),
  (0, 48),
  (0, 49),
  (0, 50),
  (0, 51),
  (0, 52),
  (0, 53),
  (0, 54),
  (0, 55),
  (0, 56),
  (0, 57),
  (0, 58),
  (0, 59),
  (0, 60),
  (0, 61),
  (0, 62),
  (0, 63),
  (0, 64),
  (0, 65),
  (0, 66),
  (0, 67),
  (0, 68),
  (0, 69),
  (0, 70),
  (0, 71),
  (0, 72),
  (0, 73),
  (0, 74),
  (0, 75),
  (0, 76),
  (0, 77),
  (0, 78),
  (0, 79),
  (0, 80),
  (0, 81),
  (0, 82),
  (0, 83),
  (0, 84),
  (0, 85),
  (0, 86),
  (0, 87),
  (0, 88),
  (0, 89),
  (0,

In [303]:
#for m in w.moths[0]:
#    print(m.age)
print(len(w.moths[0]))

200


In [13]:
# number of creatures
# lifespan
# initial lifespan distr
# gender distr
# 

In [41]:
np.random.uniform() < 0.5

True

In [264]:
a = [(0,2) for i in range(5)]

In [265]:
a

[(0, 2), (0, 2), (0, 2), (0, 2), (0, 2)]

In [268]:
a[0][0]

0